In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

In [2]:
#Required text pre-processing libraries are imported
import string
import nltk
import re

# download the stopwords and wordnet corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
# import tokenize from nltk library
from nltk import tokenize
# import WordNetLemmatizer from nltk library
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from nltk.probability import FreqDist

#Required data visualisation libraries are imported
import plotly.express as px
import seaborn as sns 
import matplotlib.pyplot as plt


#Required prediction modelling libraries are imported
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, precision_recall_curve, auc, roc_curve, accuracy_score, recall_score, classification_report, f1_score, precision_score, precision_recall_fscore_support, roc_auc_score, make_scorer
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

import xgboost
from xgboost import XGBClassifier


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jillian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jillian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jillian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jillian/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
tweets=pd.read_csv('./data/tweet_dataset.csv')

In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
tweets

,id,created_at,author_id,text,public_metrics,reply_settings,lang,source,conversation_id,in_reply_to_user_id,referenced_tweets,geo
0,1559284907788083204,2022-08-15T21:04:26.000Z,1525515243912298499,Corinna Brown - Short twill skirt in black/white checked by @hm - https://t.co/sDY2CJMMWV https://t.co/ZDirTLevCR,"{'retweet_count': 1, 'reply_count': 0, 'like_count': 4, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559284907788083204,NaN,NaN,NaN
1,1559270690381086721,2022-08-15T20:07:57.000Z,912295575697203201,@hm when will the pants shown here be available online? https://t.co/pxlPkhUcKr,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559270690381086721,1.439948e+07,NaN,NaN
2,1559246999165206530,2022-08-15T18:33:48.000Z,1549124854812639232,#outfitoftheday @Macys jewelry @Coach handbag @hm crochet top &amp; pants https://t.co/csuJfZSwvF,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for Android,1559246999165206530,NaN,NaN,NaN
3,1559185096284016641,2022-08-15T14:27:49.000Z,1443595240137449472,"Hot tip: @hm has a Garment Collection program where if you bring in old clothes of any condition/brand, you can get a discount (usually ~15%) coupon for your next purchase. Great way to recycle clothes that aren't eligible for donation!","{'retweet_count': 1, 'reply_count': 0, 'like_count': 3, 'quote_count': 0}",everyone,en,Twitter Web App,1559185096284016641,NaN,NaN,NaN
4,1559123491324309504,2022-08-15T10:23:02.000Z,2283241081,My sister who loves @hm shops every month from the store here and has taken me along a bunch of times. \nI’ve always faced problems with finding pants my size and this time I just had to do something about it. \n#plussizefashion #midsize #sizeinclusivity,"{'retweet_count': 0, 'reply_count': 1, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559122718385385472,2.283241e+09,"[{'type': 'replied_to', 'id': '1559122718385385472'}]",NaN
5,1558984167085355008,2022-08-15T01:09:24.000Z,2426396108,3/ We gladly accept hand-me-ups from family &amp; friends these days due to rising charity &amp; tip shop textile prices. Free clothes are the way to go when there is so much textile waste - we're not shopping to keep it out of landfill at those prices. This @hm coat was our daughters. https://t.co/jjkWHAglO6,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter Web App,1558977104275918848,2.426396e+09,"[{'type': 'replied_to', 'id': '1558981357530845184'}]",NaN
6,1558872612540907520,2022-08-14T17:46:07.000Z,87229885,"Here for linen, fresh whites and sandals 🕊\n\nshirt: @ukgap\nvest: @hm\ntrousers: @andotherstories\nsandals: @dune_london \nbag: @amazonfashioneu @ Manchester, United Kingdom https://t.co/TIgXs1k0z0","{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Instagram,1558872612540907520,NaN,NaN,"{'place_id': '315b740b108481f6', 'coordinates': {'type': 'Point', 'coordinates': [-2.2463, 53.4793]}}"
7,1558830828259385344,2022-08-14T15:00:05.000Z,419663786,"Life's a party, so dress for it!\n\nShirt, pants, sandals: @HM\nRing, watch: @Maisonsimons\n\n📸: @Tracey.jazmin⁣⁣⁣⁣⁣⁣⁣⁣\nModel: @lancedugay⁣⁣⁣⁣⁣⁣\n📍: Glenora building – Former @royal_alberta_museum https://t.co/CVxV2dczOl","{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Emplifi,1558830828259385344,NaN,NaN,NaN
8,1558765315416174596,2022-08-14T10:39:46.000Z,1557223542625669121,Won the contest of the prettiest chick wearing @hm cute fits ! \nFashion styling Day2Night dress. Nude colour casual dress truned into a sexy night club dress . Fashion Boss 💜 https://t.co/aCKc9GJh3j,"{'retweet_count': 5, 'reply_count': 1, 'like_count': 13, 'quote_count': 0}",everyone,en,Twitter for Android,1558765315416174596,NaN,NaN,NaN
9,1558647641663279108,2022-08-14T02:52:10.000Z,321152335,@hm your baby clothes are going to be the reason I have to go back to

In [6]:
#need general preprocessing function to clean my tweets--lowercase, remove punctuation, remove emojis, etc.

In [7]:
#installing tweet-preprocessor
!pip install tweet-preprocessor

In [8]:
#adding the cleaned (After removal of URLs, Mentions) tweets to a new column as a new feature ‘clean_text’

In [9]:
import preprocessor as p
#forming a separate feature for cleaned tweets
for i,v in enumerate(tweets['text']):
    tweets.loc[i,'text']  = p.clean(v)

In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
tweets.head()

,id,created_at,author_id,text,public_metrics,reply_settings,lang,source,conversation_id,in_reply_to_user_id,referenced_tweets,geo
0,1559284907788083204,2022-08-15T21:04:26.000Z,1525515243912298499,Corinna Brown - Short twill skirt in black/white checked by -,"{'retweet_count': 1, 'reply_count': 0, 'like_count': 4, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559284907788083204,NaN,NaN,NaN
1,1559270690381086721,2022-08-15T20:07:57.000Z,912295575697203201,when will the pants shown here be available online?,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559270690381086721,1.439948e+07,NaN,NaN
2,1559246999165206530,2022-08-15T18:33:48.000Z,1549124854812639232,jewelry handbag crochet top &amp; pants,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for Android,1559246999165206530,NaN,NaN,NaN
3,1559185096284016641,2022-08-15T14:27:49.000Z,1443595240137449472,"Hot tip: has a Garment Collection program where if you bring in old clothes of any condition/brand, you can get a discount (usually ~15%) coupon for your next purchase. Great way to recycle clothes that aren't eligible for donation!","{'retweet_count': 1, 'reply_count': 0, 'like_count': 3, 'quote_count': 0}",everyone,en,Twitter Web App,1559185096284016641,NaN,NaN,NaN
4,1559123491324309504,2022-08-15T10:23:02.000Z,2283241081,My sister who loves shops every month from the store here and has taken me along a bunch of times. Ive always faced problems with finding pants my size and this time I just had to do something about it.,"{'retweet_count': 0, 'reply_count': 1, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559122718385385472,2.283241e+09,"[{'type': 'replied_to', 'id': '1559122718385385472'}]",NaN


In [12]:
#important libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package wordnet to /Users/jillian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jillian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#Remove Digits and lower the text (makes it easy to deal with)
tweets['text_clean'] = tweets['text'].astype(str).str.replace('\d+', '')
tweets['text_clean'] = tweets['text_clean'].str.lower()

In [14]:
tweets.head()

,id,created_at,author_id,text,public_metrics,reply_settings,lang,source,conversation_id,in_reply_to_user_id,referenced_tweets,geo,text_clean
0,1559284907788083204,2022-08-15T21:04:26.000Z,1525515243912298499,Corinna Brown - Short twill skirt in black/white checked by -,"{'retweet_count': 1, 'reply_count': 0, 'like_count': 4, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559284907788083204,NaN,NaN,NaN,corinna brown - short twill skirt in black/white checked by -
1,1559270690381086721,2022-08-15T20:07:57.000Z,912295575697203201,when will the pants shown here be available online?,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559270690381086721,1.439948e+07,NaN,NaN,when will the pants shown here be available online?
2,1559246999165206530,2022-08-15T18:33:48.000Z,1549124854812639232,jewelry handbag crochet top &amp; pants,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for Android,1559246999165206530,NaN,NaN,NaN,jewelry handbag crochet top &amp; pants
3,1559185096284016641,2022-08-15T14:27:49.000Z,1443595240137449472,"Hot tip: has a Garment Collection program where if you bring in old clothes of any condition/brand, you can get a discount (usually ~15%) coupon for your next purchase. Great way to recycle clothes that aren't eligible for donation!","{'retweet_count': 1, 'reply_count': 0, 'like_count': 3, 'quote_count': 0}",everyone,en,Twitter Web App,1559185096284016641,NaN,NaN,NaN,"hot tip: has a garment collection program where if you bring in old clothes of any condition/brand, you can get a discount (usually ~%) coupon for your next purchase. great way to recycle clothes that aren't eligible for donation!"
4,1559123491324309504,2022-08-15T10:23:02.000Z,2283241081,My sister who loves shops every month from the store here and has taken me along a bunch of times. Ive always faced problems with finding pants my size and this time I just had to do something about it.,"{'retweet_count': 0, 'reply_count': 1, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559122718385385472,2.283241e+09,"[{'type': 'replied_to', 'id': '1559122718385385472'}]",NaN,my sister who loves shops every month from the store here and has taken me along a bunch of times. ive always faced problems with finding pants my size and this time i just had to do something about it.


In [15]:
#remove punctuation
def punctuation_removal(punc):
    rem_punc = [i for i in punc if i not in string.punctuation]
    after_punc = ''.join(rem_punc)
    return after_punc

In [16]:
tweets['text_clean'] = tweets.text_clean.apply(punctuation_removal)

In [17]:
tweets.head()

,id,created_at,author_id,text,public_metrics,reply_settings,lang,source,conversation_id,in_reply_to_user_id,referenced_tweets,geo,text_clean
0,1559284907788083204,2022-08-15T21:04:26.000Z,1525515243912298499,Corinna Brown - Short twill skirt in black/white checked by -,"{'retweet_count': 1, 'reply_count': 0, 'like_count': 4, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559284907788083204,NaN,NaN,NaN,corinna brown short twill skirt in blackwhite checked by
1,1559270690381086721,2022-08-15T20:07:57.000Z,912295575697203201,when will the pants shown here be available online?,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559270690381086721,1.439948e+07,NaN,NaN,when will the pants shown here be available online
2,1559246999165206530,2022-08-15T18:33:48.000Z,1549124854812639232,jewelry handbag crochet top &amp; pants,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for Android,1559246999165206530,NaN,NaN,NaN,jewelry handbag crochet top amp pants
3,1559185096284016641,2022-08-15T14:27:49.000Z,1443595240137449472,"Hot tip: has a Garment Collection program where if you bring in old clothes of any condition/brand, you can get a discount (usually ~15%) coupon for your next purchase. Great way to recycle clothes that aren't eligible for donation!","{'retweet_count': 1, 'reply_count': 0, 'like_count': 3, 'quote_count': 0}",everyone,en,Twitter Web App,1559185096284016641,NaN,NaN,NaN,hot tip has a garment collection program where if you bring in old clothes of any conditionbrand you can get a discount usually coupon for your next purchase great way to recycle clothes that arent eligible for donation
4,1559123491324309504,2022-08-15T10:23:02.000Z,2283241081,My sister who loves shops every month from the store here and has taken me along a bunch of times. Ive always faced problems with finding pants my size and this time I just had to do something about it.,"{'retweet_count': 0, 'reply_count': 1, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559122718385385472,2.283241e+09,"[{'type': 'replied_to', 'id': '1559122718385385472'}]",NaN,my sister who loves shops every month from the store here and has taken me along a bunch of times ive always faced problems with finding pants my size and this time i just had to do something about it


In [18]:
#utilizing same lemmatize/tokenize code from initial data cleaning

In [19]:
# Instantiate a word net lemmatizer object and create helper function to apply it to the dataframe.
lemmatizer = WordNetLemmatizer()

def tokenize_lem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as its own token
    tokens = [word.lower() for sent in tokenize.sent_tokenize(text) for word in tokenize.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return [lemmatizer.lemmatize(token) for token in filtered_tokens]

In [20]:
tweets['text_clean'] = tweets.text_clean.apply(tokenize_lem)

In [21]:
tweets.head()

,id,created_at,author_id,text,public_metrics,reply_settings,lang,source,conversation_id,in_reply_to_user_id,referenced_tweets,geo,text_clean
0,1559284907788083204,2022-08-15T21:04:26.000Z,1525515243912298499,Corinna Brown - Short twill skirt in black/white checked by -,"{'retweet_count': 1, 'reply_count': 0, 'like_count': 4, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559284907788083204,NaN,NaN,NaN,"[corinna, brown, short, twill, skirt, in, blackwhite, checked, by]"
1,1559270690381086721,2022-08-15T20:07:57.000Z,912295575697203201,when will the pants shown here be available online?,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559270690381086721,1.439948e+07,NaN,NaN,"[when, will, the, pant, shown, here, be, available, online]"
2,1559246999165206530,2022-08-15T18:33:48.000Z,1549124854812639232,jewelry handbag crochet top &amp; pants,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for Android,1559246999165206530,NaN,NaN,NaN,"[jewelry, handbag, crochet, top, amp, pant]"
3,1559185096284016641,2022-08-15T14:27:49.000Z,1443595240137449472,"Hot tip: has a Garment Collection program where if you bring in old clothes of any condition/brand, you can get a discount (usually ~15%) coupon for your next purchase. Great way to recycle clothes that aren't eligible for donation!","{'retweet_count': 1, 'reply_count': 0, 'like_count': 3, 'quote_count': 0}",everyone,en,Twitter Web App,1559185096284016641,NaN,NaN,NaN,"[hot, tip, ha, a, garment, collection, program, where, if, you, bring, in, old, clothes, of, any, conditionbrand, you, can, get, a, discount, usually, coupon, for, your, next, purchase, great, way, to, recycle, clothes, that, arent, eligible, for, donation]"
4,1559123491324309504,2022-08-15T10:23:02.000Z,2283241081,My sister who loves shops every month from the store here and has taken me along a bunch of times. Ive always faced problems with finding pants my size and this time I just had to do something about it.,"{'retweet_count': 0, 'reply_count': 1, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559122718385385472,2.283241e+09,"[{'type': 'replied_to', 'id': '1559122718385385472'}]",NaN,"[my, sister, who, love, shop, every, month, from, the, store, here, and, ha, taken, me, along, a, bunch, of, time, ive, always, faced, problem, with, finding, pant, my, size, and, this, time, i, just, had, to, do, something, about, it]"


In [22]:
#removing stop words

In [23]:
sw = stopwords.words('english')

# Helper function to remove stopwords
def remove_stopwords(token_list):
    """
    Given a list of tokens, return a list where the tokens
    that are also present in sw have been removed
    """
    stopwords_r = [token for token in token_list if token not in sw]
    return stopwords_r

In [24]:
# Apply the removal function to df.clean
tweets.text_clean=tweets.text_clean.apply(remove_stopwords)

# create new column from df_clean that converts list of tokens to a string.
tweets['string']=tweets.text_clean.str.join(" ")

In [25]:
tweets.head()

,id,created_at,author_id,text,public_metrics,reply_settings,lang,source,conversation_id,in_reply_to_user_id,referenced_tweets,geo,text_clean,string
0,1559284907788083204,2022-08-15T21:04:26.000Z,1525515243912298499,Corinna Brown - Short twill skirt in black/white checked by -,"{'retweet_count': 1, 'reply_count': 0, 'like_count': 4, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559284907788083204,NaN,NaN,NaN,"[corinna, brown, short, twill, skirt, blackwhite, checked]",corinna brown short twill skirt blackwhite checked
1,1559270690381086721,2022-08-15T20:07:57.000Z,912295575697203201,when will the pants shown here be available online?,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559270690381086721,1.439948e+07,NaN,NaN,"[pant, shown, available, online]",pant shown available online
2,1559246999165206530,2022-08-15T18:33:48.000Z,1549124854812639232,jewelry handbag crochet top &amp; pants,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for Android,1559246999165206530,NaN,NaN,NaN,"[jewelry, handbag, crochet, top, amp, pant]",jewelry handbag crochet top amp pant
3,1559185096284016641,2022-08-15T14:27:49.000Z,1443595240137449472,"Hot tip: has a Garment Collection program where if you bring in old clothes of any condition/brand, you can get a discount (usually ~15%) coupon for your next purchase. Great way to recycle clothes that aren't eligible for donation!","{'retweet_count': 1, 'reply_count': 0, 'like_count': 3, 'quote_count': 0}",everyone,en,Twitter Web App,1559185096284016641,NaN,NaN,NaN,"[hot, tip, ha, garment, collection, program, bring, old, clothes, conditionbrand, get, discount, usually, coupon, next, purchase, great, way, recycle, clothes, arent, eligible, donation]",hot tip ha garment collection program bring old clothes conditionbrand get discount usually coupon next purchase great way recycle clothes arent eligible donation
4,1559123491324309504,2022-08-15T10:23:02.000Z,2283241081,My sister who loves shops every month from the store here and has taken me along a bunch of times. Ive always faced problems with finding pants my size and this time I just had to do something about it.,"{'retweet_count': 0, 'reply_count': 1, 'like_count': 0, 'quote_count': 0}",everyone,en,Twitter for iPhone,1559122718385385472,2.283241e+09,"[{'type': 'replied_to', 'id': '1559122718385385472'}]",NaN,"[sister, love, shop, every, month, store, ha, taken, along, bunch, time, ive, always, faced, problem, finding, pant, size, time, something]",sister love shop every month store ha taken along bunch time ive always faced problem finding pant size time something


In [26]:
final_tweets = tweets['string']

In [27]:
#ready to run my tweets through my model to predict proba!

In [28]:
#loading in dataset to train my final model

In [29]:
df = pd.read_csv('./data/singe_word_data.csv')

In [31]:
# Specify X as the cleaned strings in df and y as the target-Positive_Rating.
X = df['String']
y = df['Positive_Rating']

In [32]:
#Performed the train-test split, using 20% for the hold-out data.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.20)

In [33]:
best_model2 = ImPipeline([
           ('vect', TfidfVectorizer(max_df=.5, max_features=3000, ngram_range=(1,2))),
           ('lr', LogisticRegression(random_state=42, class_weight=None, solver='saga')),
])

In [34]:
best_model2.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.5, max_features=3000,
                                 ngram_range=(1, 2))),
                ('lr', LogisticRegression(random_state=42, solver='saga'))])

In [35]:
best_model2.predict(final_tweets)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [36]:
best_model2.predict_proba(final_tweets)

array([[0.30548426, 0.69451574],
       [0.22827892, 0.77172108],
       [0.15195288, 0.84804712],
       [0.21780663, 0.78219337],
       [0.19850292, 0.80149708],
       [0.39197956, 0.60802044],
       [0.14760478, 0.85239522],
       [0.12524972, 0.87475028],
       [0.07568916, 0.92431084],
       [0.55885261, 0.44114739],
       [0.44941971, 0.55058029],
       [0.27119498, 0.72880502],
       [0.18831656, 0.81168344],
       [0.47673175, 0.52326825],
       [0.15743333, 0.84256667],
       [0.40470999, 0.59529001],
       [0.2521529 , 0.7478471 ],
       [0.35035715, 0.64964285],
       [0.82950624, 0.17049376],
       [0.05374673, 0.94625327]])

In [37]:
#IT WORKED!

In [38]:
final_tweets

0                                                                                                                                   corinna brown short twill skirt blackwhite checked
1                                                                                                                                                          pant shown available online
2                                                                                                                                                 jewelry handbag crochet top amp pant
3                   hot tip ha garment collection program bring old clothes conditionbrand get discount usually coupon next purchase great way recycle clothes arent eligible donation
4                                                               sister love shop every month store ha taken along bunch time ive always faced problem finding pant size time something
5     gladly accept handmeups family amp friend day due rising charity amp tip shop t

In [39]:
df = pd.DataFrame(best_model2.predict(final_tweets))

In [40]:
df

,0
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,0


In [41]:
df2=pd.DataFrame(final_tweets)

In [42]:
df2

,string
0,corinna brown short twill skirt blackwhite checked
1,pant shown available online
2,jewelry handbag crochet top amp pant
3,hot tip ha garment collection program bring old clothes conditionbrand get discount usually coupon next purchase great way recycle clothes arent eligible donation
4,sister love shop every month store ha taken along bunch time ive always faced problem finding pant size time something
5,gladly accept handmeups family amp friend day due rising charity amp tip shop textile price free clothes way go much textile waste shopping keep landfill price coat wa daughter
6,linen fresh white sandal shirt bag manchester united kingdom
7,life party dress itshirt pant sandal watch jazminmodel glenora building former
8,contest prettiest chick wearing cute fit fashion styling daynight dress nude colour casual dress truned sexy night club dress fashion bos
9,baby clothes going reason go back work stay home mom


In [46]:
prediction_df = df.join(df2)

In [47]:
prediction_df

,0,string
0,1,corinna brown short twill skirt blackwhite checked
1,1,pant shown available online
2,1,jewelry handbag crochet top amp pant
3,1,hot tip ha garment collection program bring old clothes conditionbrand get discount usually coupon next purchase great way recycle clothes arent eligible donation
4,1,sister love shop every month store ha taken along bunch time ive always faced problem finding pant size time something
5,1,gladly accept handmeups family amp friend day due rising charity amp tip shop textile price free clothes way go much textile waste shopping keep landfill price coat wa daughter
6,1,linen fresh white sandal shirt bag manchester united kingdom
7,1,life party dress itshirt pant sandal watch jazminmodel glenora building former
8,1,contest prettiest chick wearing cute fit fashion styling daynight dress nude colour casual dress truned sexy night club dress fashion bos
9,0,baby clothes going reason go back work stay home mom


In [53]:
#checking the full tweet for tweets predicted as negatively sentimented 

In [52]:
tweets['text'][9]

'your baby clothes are going to be the reason I have to go back to work and not be a stay at home mom'

In [55]:
#^^actually positive sentiment

In [54]:
tweets['text'][18]

'shirt arriving creased to hell is one thing but burn marks on front and back? Immediate return'

In [56]:
#^^correctly identified as negative

In [34]:
#feed tweets to my prepocessing function that matches the steps I took for my final model

In [35]:
#might have to create more generalized preprocessing with regex for the Tweets to be more generalizable

In [36]:
#in presentation discuss how this is generalizable to Instagram, discussion boards, Reddit, Discord, YouTube reviews etc. etc. 

In [37]:
#use for promo boxes/unboxing/clothing haul

In [38]:
#different price points reflected on different forums

In [39]:
#pipeline contains vectorizer (that was trained with training data)

In [40]:
#run my tweets and do a .predict (probably the ones I can predict with my own eyes)

In [41]:
#.predictproba for BEST retweetable (depending on model)

In [42]:
#pickling is deploying in app

In [43]:
#potential limitations: disproportion of positive reviews but people may disproportionately go online to complain (disproportionate negative reviews)